In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import os
annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Limenitis_camilla
Exon_1 Match
[['Vanessa_tameamea', ['6304231', '6304319']], ['Nymphalis_io', ['6304231', '6304319']], ['Danaus_plexippus', ['6304231', '6304319']], ['Heliconius_melpomene', ['6304231', '6304319']]]
Exon_2 Match
[['Vanessa_tameamea', ['6304395', '6304484']], ['Nymphalis_io', ['6304395', '6304484']], ['Danaus_plexippus', ['6304395', '6304484']], ['Heliconius_melpomene', ['6304395', '6304484']]]
Exon_3 mismatch
[['Vanessa_tameamea', ['6304950', '6305089']], ['Nymphalis_io', ['6304950', '6305212']], ['Danaus_plexippus', ['6304944', '6305089']], ['Heliconius_melpomene', ['6304944', '6305089']]]
Proceed?y
Vanessa_tameamea Exon_3 20_frame0 1.38 
NQENRIKDKEVKKKKKMLIDIQSTGVEKENEWPLPPVLEAPKEDKD
Nymphalis_io Exon_3 20_frame0 1.514 
NQENRIKDKEVKKKKKMLIDIQSTGVEKENEWPLPPVLEAPKEDKDGKADFDFLQNSLFFVMLFIDKVYDLLAFRCTIYITRLHNVG
Danaus_plexippus Exon_3 20_frame0 1.405 
SSNQENRIKDKEVKKKKKMLIDIQSTGVEKENEWPLPPVLEAPKEDKD
Heliconius_melpomene Exon_3 20_frame0 0.992 
SSNQENRIKDKEVKKKKKMLIDIQSTGV

Choose Species 1 - 44
Exon_13 Match
[['Vanessa_tameamea', ['6312900', '6313015']], ['Nymphalis_io', ['6312900', '6313015']], ['Danaus_plexippus', ['6312900', '6313015']], ['Heliconius_melpomene', ['6312900', '6313015']]]
Exon_14 mismatch
[['Vanessa_tameamea', ['6313482', '6313571']], ['Nymphalis_io', ['6313482', '6313571']], ['Danaus_plexippus', ['6313482', '6313583']], ['Heliconius_melpomene', ['6313482', '6313571']]]
Proceed?y
Vanessa_tameamea Exon_14 47_frame0 1.145 
FFKSHEPYEEYDFDSGGGAWKGITMNNND
Nymphalis_io Exon_14 47_frame0 1.466 
FFKSHEPYEEYDFDSGGGAWKGITMNNND
Danaus_plexippus Exon_14 47_frame0 1.482 
FFKSHEPYEEYDFDSGGGAWKGITMNNNDRYVR
Heliconius_melpomene Exon_14 47_frame0 1.304 
FFKSHEPYEEYDFDSGGGAWKGITMNNND
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


20.Vanessa_tame --------YFDSHQPYEEYPLDD------------NKLELREAEN
set47_frame0    LTVYSHYRFFKSHEPYEEYDFDSGGGAWKGITMNNNDRYVRLL--
                        :*.**:***** :*.            *.  :*    

CLUSTAL format a

Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


20.Vanessa_tame ACRLTLLPKTGKSFNYLTRKREKIIEKNIFQACDVSIYSAEASIR
set46_frame2    Y-QIPAFPCDCQKFKLLSPRIEKILYRYILLMWLLTKL-------
                  ::. :*   :.*: *: : ***: : *:    ::         

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


29.Nymphalis_io ACK------------MTVLPKTNKSFSYLTRKREKVIEKQLHQSSDVSNSSSDTSY----
set173_frame2   ITKHIFSMVCPKEIIYSRLQATVRTSTVLTKDRTSRKTRRCENERETTRTDGESNYSSFY
                  *             : *  * :: : **:.* .   :: .:. :.:.:..::.*    

29.Nymphalis_io ---------------------------R
set173_frame2   SSFFKTESSSTTDDCDVRNRNRNSKLVS
                                            

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


13.Danaus_plexi ---------------------ACKLTVPLKTSKACRITR--ENHNQALRRDG-VNYSAGT
set173_frame2   ITKHIFSMVCPKEIIYSRLQATVRTSTVLTKDRTSRKTRRCENERETTRTDGESNYSSFY
                                     : : :. *...::.* **  **..:: * **  ***:  

13.Danaus_plexi SN-----------------